### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import datetime
from imblearn.over_sampling import SMOTE
from zipfile import ZipFile

import wrangle

In [2]:
test, df = wrangle.acquire_fraud('cc_fraud.zip')

In [3]:
test.shape,df.shape

((555719, 23), (1296675, 23))

In [4]:
df = wrangle.prep(df)
test = wrangle.prep(test)

In [5]:
df.head()

,merchant,category,amt,gender,zip,city_pop,job,merch_lat,merch_long,is_fraud,age,dayofweek,hourofday,age_group
trans_date_trans_time,,,,,,,,,,,,,,
2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,28654,3495,"Psychologist, counselling",36.011293,-82.048315,0,30,Tuesday,0,Young_Adult
2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,99160,149,Special educational needs teacher,49.159047,-118.186462,0,40,Tuesday,0,Adult
2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,83252,4154,Nature conservation officer,43.150704,-112.154481,0,56,Tuesday,0,Mid_Age
2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,59632,1939,Patent attorney,47.034331,-112.561071,0,52,Tuesday,0,Early_Mid_Age
2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,24433,99,Dance movement psychotherapist,38.674999,-78.632459,0,32,Tuesday,0,Young_Adult


In [6]:
target,features = wrangle.get_target_and_features(df)
print(target)
print('______')
print(features)

is_fraud
______
['merchant', 'category', 'amt', 'gender', 'zip', 'city_pop', 'job', 'merch_lat', 'merch_long', 'age', 'dayofweek', 'hourofday', 'age_group']


In [7]:
x_train,x_validate,y_train,y_validate = wrangle.train_val(df)

In [8]:
x_train.sample()

,merchant,category,amt,gender,zip,city_pop,job,merch_lat,merch_long,age,dayofweek,hourofday,age_group
trans_date_trans_time,,,,,,,,,,,,,
2020-04-26 22:02:41,fraud_O'Keefe-Wisoky,food_dining,61.06,M,78631,2395,"Pharmacist, community",30.247401,-99.582274,26,Sunday,22,Young_Adult


In [9]:
y_train.sample()

trans_date_trans_time
2019-03-02 18:19:01    0
Name: is_fraud, dtype: int64

In [10]:
x_test,y_test = wrangle.test_df_x_y_split(test)

In [11]:
x_test.sample()

,merchant,category,amt,gender,zip,city_pop,job,merch_lat,merch_long,age,dayofweek,hourofday,age_group
trans_date_trans_time,,,,,,,,,,,,,
2020-09-07 16:46:01,fraud_Beier LLC,entertainment,15.09,F,30426,1100,"Conservator, museum/gallery",33.512054,-82.64736,70,Monday,16,Retirement_Age


In [12]:
y_test.sample()

trans_date_trans_time
2020-10-08 19:24:51    0
Name: is_fraud, dtype: int64